# BioMedGraphica Drug

## 1. Data Access  
### Direct Download Links  
**PubChem**: Can be downloaded directly via the link without the need for registration. [Link1](https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PubChem_compound_cache_i8wtnP0JmLWvn5qGGP7Tr3VBFSHgUpkp4wyCZfgdkGT4BKw%22,%22where%22:{%22ands%22:[{%22input%22:{%22type%22:%22netcachekey%22,%22idtype%22:%22cid%22,%22key%22:%22i8wtnP0JmLWvn5qGGP7Tr3VBFSHgUpkp4wyCZfgdkGT4BKw%22}}]}}); [Link2](https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PubChem_compound_cache_J2CBMEv5LkUZbyx2rg5lX8Ow8tCczY2R97SW3eylhNzsvLg%22,%22where%22:{%22ands%22:[{%22input%22:{%22type%22:%22netcachekey%22,%22idtype%22:%22cid%22,%22key%22:%22J2CBMEv5LkUZbyx2rg5lX8Ow8tCczY2R97SW3eylhNzsvLg%22}}]}})  
**NDC**: Can be downloaded directly via the link without the need for registration. [Link](https://www.accessdata.fda.gov/cder/ndctext.zip)  
**UNII**: Need Registration. [Link](https://precision.fda.gov/uniisearch/archive/latest/UNII_Data.zip)  
**DrugBank**: Need Registration. [Link](https://go.drugbank.com/releases/5-1-12/downloads/all-drug-links)  

### UNII Data Fetch

In [ ]:
import pandas as pd
import json
import requests

def fetch_and_write_annotations(base_url, total_pages, file_path):
    with open(file_path, 'w') as file:
        file.write('[')
        first_entry = True 
        
        for page in range(1, total_pages + 1):
            url = f"{base_url}?page={page}"
            print(f"Fetching data from: {url}") 
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                annotations = data.get('Annotations', {}).get('Annotation', [])
                
                for annotation in annotations:
                    if not first_entry:
                        file.write(',')
                    json.dump(annotation, file)
                    first_entry = False
            else:
                print(f"Failed to retrieve data for page {page}: {response.status_code}")
                continue
        
        file.write(']')

base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/annotations/heading/UNII/JSON"
total_pages = 153 # change this to the total number of pages
file_path = "unii_data.json"

fetch_and_write_annotations(base_url, total_pages, file_path)

print(f"Data saved to {file_path}")

## 2. Load Data

### 2.1 Pubchem

In [1]:
import pandas as pd

use_columns = ['cid', 'cmpdname', 'iupacname', 'cmpdsynonym', 'inchi', 'smiles', 'inchikey']
df_pubchem1 = pd.read_csv('PubChem_compound_list_drug.csv',usecols=use_columns)
df_pubchem2 = pd.read_csv('PubChem_compound_list_Pharmacology_and_Biochemistry.csv',usecols=use_columns)

df_pubchem = pd.merge(df_pubchem1, df_pubchem2, how='outer')
df_pubchem.rename(columns={'cid':'PubChem_CID', 'smiles':'PubChem_Canonical_SMILES', 'inchi': 'InChI', 'cmpdname': 'PubChem_Name'}, inplace=True)
df_pubchem[['PubChem_CID']] = df_pubchem[['PubChem_CID']].astype(str)
df_pubchem.sort_values(by='PubChem_CID', inplace=True)
df_pubchem.reset_index(drop=True, inplace=True)
df_pubchem

,PubChem_CID,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,inchikey,iupacname
0,1,Acetyl-DL-carnitine,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,RDHQFKQIGNGIED-UHFFFAOYSA-N,3-acetyloxy-4-(trimethylazaniumyl)butanoate
1,1000,2-Amino-1-phenylethanol,2-Amino-1-phenylethanol|phenylethanolamine|756...,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",C1=CC=C(C=C1)C(CN)O,ULSIYEODSMZIPX-UHFFFAOYSA-N,2-amino-1-phenylethanol
2,100001,N-Benzoylhistidine,N-Benzoylhistidine|benzoylhistidine|3-(1H-Imid...,InChI=1S/C13H13N3O3/c17-12(9-4-2-1-3-5-9)16-11...,C1=CC=C(C=C1)C(=O)NC(CC2=CN=CN2)C(=O)O,AUDPUFBIVWMAED-UHFFFAOYSA-N,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid
3,10001145,Shoyuflavone B,Shoyuflavone B|190712-88-0|2-hydroxy-3-[5-hydr...,InChI=1S/C19H14O10/c20-9-3-1-8(2-4-9)11-7-28-1...,C1=CC(=CC=C1C2=COC3=CC(=CC(=C3C2=O)O)OC(C(C(=O...,YZECNQRIFYQRPI-UHFFFAOYSA-N,2-hydroxy-3-[5-hydroxy-3-(4-hydroxyphenyl)-4-o...
4,10001250,1-Methyl-3-[2-[4-(3-methoxyphenyl)butyl]phenox...,SCHEMBL8703946|DDWAVUOYXCFCBW-UHFFFAOYSA-N|1-M...,InChI=1S/C24H33NO2/c1-25-16-8-11-21(18-25)19-2...,CN1CCCC(C1)COC2=CC=CC=C2CCCCC3=CC(=CC=C3)OC,DDWAVUOYXCFCBW-UHFFFAOYSA-N,3-[[2-[4-(3-methoxyphenyl)butyl]phenoxy]methyl...
...,...,...,...,...,...,...,...
123352,9996712,"2,6-Dimethyl-4-(3-nitrophenyl)pyridine-3,5-dic...","64603-72-1|5-methoxycarbonyl-2,6-dimethyl-4-(3...",InChI=1S/C16H14N2O6/c1-8-12(15(19)20)14(13(9(2...,CC1=C(C(=C(C(=N1)C)C(=O)OC)C2=CC(=CC=C2)[N+](=...,BXLPPWLKFWCWOY-UHFFFAOYSA-N,"5-methoxycarbonyl-2,6-dimethyl-4-(3-nitropheny..."
123353,99973,Castalin,"Castalin|19086-75-0|BA7JCC4U52|7,8,9,12,13,14,...",InChI=1S/C27H20O18/c28-2-5-14(31)23-24-20(37)1...,C1=C2C(=C(C(=C1O)O)O)C3=C4C(=C(C(=C3O)O)O)C5=C...,PPUHUWSVCUJGTD-UHFFFAOYSA-N,"7,8,9,12,13,14,17,18,19,25,29-undecahydroxy-24..."
123354,9999,Cyanogen,CYANOGEN|Oxalonitrile|Ethanedinitrile|Dicyan|C...,InChI=1S/C2N2/c3-1-2-4,C(#N)C#N,JMANVNJQNLATNU-UHFFFAOYSA-N,oxalonitrile
123355,99990161,PIMS(O2)M (N-methylsulfonylmethyl phtalimide),DTXSID401028455|NS00067647|PIMS(O2)M (N-methyl...,"InChI=1S/C10H9NO4S/c1-16(14,15)6-11-9(12)7-4-2...",CS(=O)(=O)CN1C(=O)C2=CC=CC=C2C1=O,DDKXBPDRLYOKTB-UHFFFAOYSA-N,"2-(methylsulfonylmethyl)isoindole-1,3-dione"


### 2.2 National Drug Code Directory(NDC)

In [2]:
df_ndc = pd.read_csv('product.txt', sep='\t', encoding='ISO-8859-1')

df_ndc_filter = df_ndc[['PRODUCTNDC', 'SUBSTANCENAME']]
df_ndc_filter['SUBSTANCENAME'] = df_ndc_filter['SUBSTANCENAME'].str.split(';')
df_ndc_filter = df_ndc_filter.explode('SUBSTANCENAME')
df_ndc_filter.drop_duplicates(inplace=True)
df_ndc_filter.reset_index(drop=True, inplace=True)
df_ndc_filter.dropna(subset=['SUBSTANCENAME'], inplace=True)
df_ndc_filter.dropna(subset=['PRODUCTNDC'], inplace=True)

df_ndc_filter = df_ndc_filter.groupby('SUBSTANCENAME').agg({
    'PRODUCTNDC': lambda x: ';'.join(x)
}).reset_index()
df_ndc_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_29836\1032876701.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ndc_filter['SUBSTANCENAME'] = df_ndc_filter['SUBSTANCENAME'].str.split(';')


,SUBSTANCENAME,PRODUCTNDC
0,.ALPHA.-LIPOIC ACID,17089-075;17089-304;17089-463;43742-1561;62185...
1,.ALPHA.-TOCOPHEROL,43742-1561
2,".ALPHA.-TOCOPHEROL ACETATE, DL-",17089-450
3,.BETA.-CAROTENE,13925-116;13925-117;50090-0616;75854-314
4,".BETA.-CITRONELLOL, (R)-",83021-187
...,...,...
8120,ZOLMITRIPTAN,16571-803;16571-804;27241-021;27241-022;45802-...
8121,ZOLPIDEM TARTRATE,0024-5401;0024-5421;0024-5501;0024-5521;0037-6...
8122,ZONISAMIDE,0615-8266;29300-428;29300-429;29300-430;50268-...
8123,ZUCCHINI,0268-6219


### 2.3 UNII

Data from PubChem

In [3]:
import json

with open('unii_data.json', 'r') as file:
    d_unii_data = json.load(file)

df_UNII = pd.json_normalize(d_unii_data)
df_UNII.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152870 entries, 0 to 152869
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   SourceName         152870 non-null  object
 1   SourceID           152870 non-null  object
 2   Name               152870 non-null  object
 3   Description        152870 non-null  object
 4   URL                152870 non-null  object
 5   LicenseNote        152870 non-null  object
 6   LicenseURL         152870 non-null  object
 7   Data               152870 non-null  object
 8   ANID               152870 non-null  int64 
 9   LinkedRecords.SID  152727 non-null  object
 10  LinkedRecords.CID  112090 non-null  object
dtypes: int64(1), object(10)
memory usage: 12.8+ MB


In [4]:
df_UNII = df_UNII[['SourceID', 'Name','LinkedRecords.SID', 'LinkedRecords.CID']]
df_UNII.rename(columns={'SourceID':'UNII', 'LinkedRecords.SID':'PubChem_SID', 'LinkedRecords.CID':'PubChem_CID'}, inplace=True)
df_UNII['PubChem_CID'] = df_UNII['PubChem_CID'].astype(str)
df_UNII['PubChem_CID'] = df_UNII['PubChem_CID'].str.strip('[]').str.split(',')
df_UNII = df_UNII.explode('PubChem_CID')

df_UNII['PubChem_SID'] = df_UNII['PubChem_SID'].astype(str)
df_UNII['PubChem_SID'] = df_UNII['PubChem_SID'].str.strip('[]').str.split(',')
df_UNII = df_UNII.explode('PubChem_SID')
df_UNII.replace('nan', pd.NA, inplace=True)
df_UNII.drop_duplicates(inplace=True)
df_UNII

,UNII,Name,PubChem_SID,PubChem_CID
0,Z59C03N4CV,ATTALEA SPECTABILIS WHOLE,472416461,<NA>
1,X064O0Y1A4,IODIPAMIDE MEGLUMINE,175268591,636408
2,7M19191IKG,TEDUGLUTIDE,375083188,<NA>
3,UFH8559WS5,"1,3-DICHLOROACETONE",198979424,10793
4,RLV0M78S00,MANGANESE BENZOATE TETRAHYDRATE,198979423,71586873
...,...,...,...,...
152865,7D7JG7S7ZV,"4-(1,3-Benzodioxol-5-yl)-1-[2-(dibutylamino)-2...",496144084,159595
152866,APY6XCX4XA,Sodium Hydrolyzed Potato Starch Dodecenylsucci...,496144085,<NA>
152867,N9Y5G2T2T5,N'-Trityl-DL-glutamine,496144086,53442621
152868,33W7SJ9TBX,GSK-3008348,496144087,86272868


In [5]:
df_UNII_SID = df_UNII[df_UNII['PubChem_SID'].notnull()]
df_UNII_other = df_UNII[df_UNII['PubChem_SID'].isnull()]
df_UNII_CID = df_UNII_other[df_UNII_other['PubChem_CID'].notnull()]

Data from FDA

In [6]:
df_unii = pd.read_csv('UNII_Records_20Dec2024.txt', sep='\t', dtype=str)
df_unii = df_unii[['UNII', 'Display Name', 'RN', 'PUBCHEM', 'SMILES', 'INCHIKEY']]
df_unii

,UNII,Display Name,RN,PUBCHEM,SMILES,INCHIKEY
0,0001H6R5H1,CEROUS SALICYLATE,526-17-0,76966289,[Ce+3].c1(ccccc1O)C([O-])=O.c1(ccccc1O)C([O-])...,RBJPAJHTYHKKTB-UHFFFAOYSA-K
1,000360VJE1,DI(DEHYDROABIETYL)AMINE ACETATE,53404-27-6,76969106,C[C@@]12c3ccc(C(C)C)cc3CC[C@]1([C@](CCC2)(C)CN...,SETIUTJHVNMKFM-TUICDNFPSA-N
2,0005633KTU,SYMPHYOTRICHUM OBLONGIFOLIUM WHOLE,NaN,NaN,NaN,NaN
3,000705ZASD,ADECATUMUMAB,503605-66-1,NaN,NaN,NaN
4,00072J7XWS,GERMANIUM,7440-56-4,6326954,[Ge],GNPVGFCGXDBREM-UHFFFAOYSA-N
...,...,...,...,...,...,...
159371,ZZW95F4360,DIPOTASSIUM GLUCOSE-6-PHOSPHATE,5996-17-8,111126,[K+].O(P([O-])([O-])=O)C[C@@H](O)[C@@H](O)[C@H...,BWHWCIODKVRLNE-FAOVPRGRSA-L
159372,ZZW9CP4P2Z,SUNBURST ORANGE BETTA TRANSGENIC WHOLE (OBS201...,NaN,NaN,NaN,NaN
159373,ZZY773V99Q,ALOYSIA WRIGHTII WHOLE,NaN,NaN,NaN,NaN
159374,ZZY9N8B3WA,HEMAGGLUTININ PREPROTEIN (INFLUENZA A VIRUS (A...,NaN,NaN,NaN,NaN


Combine both UNII Data

In [7]:
def merge_string_columns(df, columns, merge_name, separator=' | '):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                combined.update(row[column].split(separator))
        return separator.join(combined)
    
    # Apply the function to each row and create a new column
    combined_column_name = merge_name
    df[combined_column_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

df_unii_final = pd.merge(df_UNII, df_unii, how='outer', left_on='UNII', right_on='UNII')
df_unii_final = merge_string_columns(df_unii_final, ['Name', 'Display Name'], 'UNII_Name')
df_unii_final

,UNII,PubChem_SID,PubChem_CID,RN,PUBCHEM,SMILES,INCHIKEY,UNII_Name
0,0001H6R5H1,198967581,76966289,526-17-0,76966289,[Ce+3].c1(ccccc1O)C([O-])=O.c1(ccccc1O)C([O-])...,RBJPAJHTYHKKTB-UHFFFAOYSA-K,CEROUS SALICYLATE
1,000360VJE1,198977524,76969106,53404-27-6,76969106,C[C@@]12c3ccc(C(C)C)cc3CC[C@]1([C@](CCC2)(C)CN...,SETIUTJHVNMKFM-TUICDNFPSA-N,DI(DEHYDROABIETYL)AMINE ACETATE
2,0005633KTU,472413960,<NA>,NaN,NaN,NaN,NaN,SYMPHYOTRICHUM OBLONGIFOLIUM WHOLE
3,000705ZASD,472405886,<NA>,503605-66-1,NaN,NaN,NaN,ADECATUMUMAB
4,00072J7XWS,<NA>,6326954,7440-56-4,6326954,[Ge],GNPVGFCGXDBREM-UHFFFAOYSA-N,GERMANIUM
...,...,...,...,...,...,...,...,...
159391,ZZW95F4360,198975766,111126,5996-17-8,111126,[K+].O(P([O-])([O-])=O)C[C@@H](O)[C@@H](O)[C@H...,BWHWCIODKVRLNE-FAOVPRGRSA-L,DIPOTASSIUM GLUCOSE-6-PHOSPHATE
159392,ZZW9CP4P2Z,474492385,<NA>,NaN,NaN,NaN,NaN,SUNBURST ORANGE BETTA TRANSGENIC WHOLE (OBS201...
159393,ZZY773V99Q,472386356,<NA>,NaN,NaN,NaN,NaN,ALOYSIA WRIGHTII WHOLE
159394,ZZY9N8B3WA,472425543,<NA>,NaN,NaN,NaN,NaN,HEMAGGLUTININ PREPROTEIN (INFLUENZA A VIRUS (A...


In [8]:
# check duplicates inside the dataframe
def merge_column(df, column1, column2, new_column):
    df[column1] = df[column1].fillna('')
    df[column2] = df[column2].fillna('')
    df[new_column] = df.apply(lambda row: f"{row[column1]} {row[column2]}".strip(), axis=1)

    expanded_rows = df[new_column].str.split(expand=True).stack().reset_index(level=1, drop=True)
    expanded_rows.name = new_column

    df = df.drop(columns=[new_column]).join(expanded_rows)
    df.drop(columns=[column1, column2], inplace=True)
    df.drop_duplicates(inplace=True)
    
    return df

df_unii_final = merge_column(df_unii_final, 'PubChem_CID', 'PUBCHEM', 'PubChem CID')
df_unii_final.replace('', pd.NA, inplace=True)
df_unii_final

,UNII,PubChem_SID,RN,SMILES,INCHIKEY,UNII_Name,PubChem CID
0,0001H6R5H1,198967581,526-17-0,[Ce+3].c1(ccccc1O)C([O-])=O.c1(ccccc1O)C([O-])...,RBJPAJHTYHKKTB-UHFFFAOYSA-K,CEROUS SALICYLATE,76966289
1,000360VJE1,198977524,53404-27-6,C[C@@]12c3ccc(C(C)C)cc3CC[C@]1([C@](CCC2)(C)CN...,SETIUTJHVNMKFM-TUICDNFPSA-N,DI(DEHYDROABIETYL)AMINE ACETATE,76969106
2,0005633KTU,472413960,NaN,NaN,NaN,SYMPHYOTRICHUM OBLONGIFOLIUM WHOLE,NaN
3,000705ZASD,472405886,503605-66-1,NaN,NaN,ADECATUMUMAB,NaN
4,00072J7XWS,<NA>,7440-56-4,[Ge],GNPVGFCGXDBREM-UHFFFAOYSA-N,GERMANIUM,6326954
...,...,...,...,...,...,...,...
159391,ZZW95F4360,198975766,5996-17-8,[K+].O(P([O-])([O-])=O)C[C@@H](O)[C@@H](O)[C@H...,BWHWCIODKVRLNE-FAOVPRGRSA-L,DIPOTASSIUM GLUCOSE-6-PHOSPHATE,111126
159392,ZZW9CP4P2Z,474492385,NaN,NaN,NaN,SUNBURST ORANGE BETTA TRANSGENIC WHOLE (OBS201...,NaN
159393,ZZY773V99Q,472386356,NaN,NaN,NaN,ALOYSIA WRIGHTII WHOLE,NaN
159394,ZZY9N8B3WA,472425543,NaN,NaN,NaN,HEMAGGLUTININ PREPROTEIN (INFLUENZA A VIRUS (A...,NaN


### 2.4 DrugBank

In [9]:
df_drugbank = pd.read_csv('drug links.csv', dtype=str)
df_drugbank = df_drugbank[['DrugBank ID', 'Name', 'PubChem Compound ID', 'PubChem Substance ID', 'CAS Number']]
df_drugbank.rename(columns={'DrugBank ID':'DrugBank_ID', 'Name':'Drug_Name', 'PubChem Compound ID':'PubChem_CID', 'PubChem Substance ID':'PubChem_SID', 'CAS Number':'CAS_Number'}, inplace=True)
df_drugbank

,DrugBank_ID,Drug_Name,PubChem_CID,PubChem_SID,CAS_Number
0,DB00001,Lepirudin,NaN,46507011,138068-37-8
1,DB00002,Cetuximab,NaN,46507042,205923-56-4
2,DB00003,Dornase alfa,NaN,46507792,143831-71-4
3,DB00004,Denileukin diftitox,NaN,46506950,173146-27-5
4,DB00005,Etanercept,NaN,46506732,185243-69-0
...,...,...,...,...,...
17425,DB19452,Exidavnemab,NaN,NaN,2438229-02-6
17426,DB19453,Imciromab pentetate,NaN,NaN,138660-99-8
17427,DB19454,Cetyl oleate,NaN,NaN,22393-86-8
17428,DB19455,Cetyl myristoleate,NaN,NaN,64660-84-0


DrugBank ID and CID

In [10]:
df_drugbank_cid = df_drugbank[df_drugbank['PubChem_CID'].notnull()]
df_drugbank_cid = df_drugbank_cid[['PubChem_CID', 'DrugBank_ID', 'Drug_Name']]
df_drugbank_cid = df_drugbank_cid.groupby('PubChem_CID').agg({
    'DrugBank_ID': lambda x: ';'.join(x),
    'Drug_Name': lambda x: ' | '.join(x),
}).reset_index()
df_drugbank_cid

,PubChem_CID,DrugBank_ID,Drug_Name
0,100016,DB03068,Zebularine
1,10007,DB01556,Chlorphentermine
2,100094,DB05779,Oglufanide
3,1001,DB04325,Phenethylamine
4,100154,DB05284,CA4P
...,...,...,...
8719,9994066,DB08134,4-[(6-chloropyrazin-2-yl)amino]benzenesulfonamide
8720,9994897,DB08754,N-Caffeoyltyramine
8721,9996708,DB01936,alpha-D-arabinofuranose
8722,9998128,DB08450,N-1H-indazol-5-yl-2-(6-methylpyridin-2-yl)quin...


DrugBank id and SID

In [11]:
df_drugbank_other = df_drugbank[df_drugbank['PubChem_CID'].isnull()]
df_drugbank_SID = df_drugbank_other[df_drugbank_other['PubChem_SID'].notnull()]
df_drugbank_SID = df_drugbank_SID[['PubChem_SID', 'DrugBank_ID', 'Drug_Name']]
df_drugbank_SID = df_drugbank_SID.groupby('PubChem_SID').agg({
    'DrugBank_ID': lambda x: ';'.join(x),
    'Drug_Name': lambda x: ' | '.join(x),
}).reset_index()
df_drugbank_SID

,PubChem_SID,DrugBank_ID,Drug_Name
0,175426991,DB05384,Carbopol 974P
1,175427018,DB05488,Technetium Tc-99m ciprofloxacin
2,175427071,DB06439,Tyloxapol
3,175427138,DB08894,Peginesatide
4,175427151,DB08913,Radium Ra 223 dichloride
...,...,...,...
1763,46509151,DB01279,Galsulfase
1764,46509198,DB01281,Abatacept
1765,99444385,DB07914,"2-hydroxy-6-oxo-6-phenyl-2,4-hexadienoic acid"
1766,99444386,DB07915,"(2E,4E)-2-Hydroxy-6-oxo-6-phenyl-2,4-hexadieno..."


Remaining DrugBank

In [12]:
df_drugbank_other = df_drugbank_other[df_drugbank_other['PubChem_SID'].isnull()]
df_drugbank_other

,DrugBank_ID,Drug_Name,PubChem_CID,PubChem_SID,CAS_Number
33,DB00035,Desmopressin,NaN,NaN,16679-58-6
4633,DB04923,rhThrombin,NaN,NaN,1270043-60-1
4766,DB05085,TM30339,NaN,NaN,NaN
5070,DB05467,Palovarotene,NaN,NaN,410528-02-8
5110,DB05513,Atiprimod,NaN,NaN,123018-47-3
...,...,...,...,...,...
17425,DB19452,Exidavnemab,NaN,NaN,2438229-02-6
17426,DB19453,Imciromab pentetate,NaN,NaN,138660-99-8
17427,DB19454,Cetyl oleate,NaN,NaN,22393-86-8
17428,DB19455,Cetyl myristoleate,NaN,NaN,64660-84-0


## 3. Data Merging

In [14]:
# check duplicates inside the dataframe
def merge_column(df, column1, column2, new_column):
    df[column1] = df[column1].fillna('')
    df[column2] = df[column2].fillna('')
    df[new_column] = df.apply(lambda row: f"{row[column1]} {row[column2]}".strip(), axis=1)

    expanded_rows = df[new_column].str.split(expand=True).stack().reset_index(level=1, drop=True)
    expanded_rows.name = new_column

    df = df.drop(columns=[new_column]).join(expanded_rows)
    df.drop(columns=[column1, column2], inplace=True)
    df.drop_duplicates(inplace=True)
    
    return df

In [15]:
def merge_string_columns(df, columns, merge_name, separator, split_values=True):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                if split_values:
                    combined.update(row[column].split(separator))
                else:
                    combined.add(row[column])
        return separator.join(combined)

    df[merge_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

### 3.1 NDC and UNII

In [16]:
# NDC's substance name uses the same format as the UNII's Substance_Name
df_unii_final.rename(columns={'UNII_Name':'SUBSTANCENAME'}, inplace=True)
df_ndc_unii = pd.merge(df_ndc_filter, df_unii_final, on='SUBSTANCENAME', how='outer')
df_ndc_unii = df_ndc_unii.rename(columns={'SMILES':'UNII_SMILES', 'SUBSTANCENAME':'UNII_Name'})
df_ndc_unii

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,RN,UNII_SMILES,INCHIKEY,PubChem CID
0,.ALPHA.-LIPOIC ACID,17089-075;17089-304;17089-463;43742-1561;62185...,NaN,NaN,NaN,NaN,NaN,NaN
1,.ALPHA.-TOCOPHEROL,43742-1561,NaN,NaN,NaN,NaN,NaN,NaN
2,".ALPHA.-TOCOPHEROL ACETATE, DL-",17089-450,NaN,NaN,NaN,NaN,NaN,NaN
3,.BETA.-CAROTENE,13925-116;13925-117;50090-0616;75854-314,NaN,NaN,NaN,NaN,NaN,NaN
4,".BETA.-CITRONELLOL, (R)-",83021-187,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
166852,β-alanyl pantothenamide | .BETA.-ALANYL PANTOT...,NaN,Y8JBU55TWT,474494092,897045-90-8,[C@H](C(NCCC(NCCC(O)=O)=O)=O)(C(CO)(C)C)O,ZEFQNPXWTQSVAH-JTQLQIEISA-N,141121107
166853,γ-4-Methylpiperidinopropyl benzoate hydrochlor...,NaN,V6C9VR2B4P,474492957,78219-44-0,C(c1ccccc1)(=O)OCCCN2CCC(CC2)C.Cl,UKBYXYHBUNXADR-UHFFFAOYSA-N,3060661
166854,γ-Thujaplicin | .GAMMA.-THUJAPLICIN,NaN,CA2K6LJ2BV,472390426,672-76-4,c1(ccc(=O)c(cc1)O)C(C)C,WKEWHSLZDDZONF-UHFFFAOYSA-N,12649
166855,"γ1-Cadinene, (+)- | .GAMMA.1-CADINENE, (+)-",NaN,LU9YQW6GXR,482554158,66141-11-5,C(C)(C)[C@@H]1[C@@]2([C@@](C(C)=CC1)(CCC(=C)C2...,COEMCKKNQLRLNH-RBSFLKMASA-N,101324803


### 3.2 Add Pubchem

In [17]:
df_ndc_unii.rename(columns={'PubChem CID':'PubChemCID'}, inplace=True)
df_ndc_unii_pubchem = pd.merge(df_ndc_unii, df_pubchem, left_on='PubChemCID', right_on='PubChem_CID', how='outer')
df_ndc_unii_pubchem = merge_column(df_ndc_unii_pubchem, 'PubChemCID', 'PubChem_CID', 'PubChem CID')
df_ndc_unii_pubchem = merge_column(df_ndc_unii_pubchem, 'INCHIKEY', 'inchikey', 'InChIKEY')
df_ndc_unii_pubchem

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,RN,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,PubChem CID,InChIKEY
0,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",NaN,07OP6H4V4A,198939309,14992-62-2,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,Acetyl-DL-carnitine,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate,1,RDHQFKQIGNGIED-UHFFFAOYSA-N
1,PHENYLETHANOLAMINE,NaN,2P4Y56479O,198959170,7568-93-6,c1ccc(cc1)C(O)CN,2-Amino-1-phenylethanol,2-Amino-1-phenylethanol|phenylethanolamine|756...,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",C1=CC=C(C=C1)C(CN)O,2-amino-1-phenylethanol,1000,ULSIYEODSMZIPX-UHFFFAOYSA-N
2,"3-CHLORO-1,1,1-TRIFLUOROPROPANE",NaN,5C535H1OII,252164000,460-35-5,C(CCCl)(F)(F)F,NaN,NaN,NaN,NaN,NaN,10000,ZPIFKCVYZBVZIV-UHFFFAOYSA-N
3,NaN,NaN,NaN,NaN,NaN,NaN,N-Benzoylhistidine,N-Benzoylhistidine|benzoylhistidine|3-(1H-Imid...,InChI=1S/C13H13N3O3/c17-12(9-4-2-1-3-5-9)16-11...,C1=CC=C(C=C1)C(=O)NC(CC2=CN=CN2)C(=O)O,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid,100001,AUDPUFBIVWMAED-UHFFFAOYSA-N
4,"N,N-DIDESMETHYLDILTIAZEM | N,N-Didesmethyldilt...",NaN,EFG5T3366P,474494223,115973-28-9,O(C(C)=O)[C@@H]1[C@@H](Sc2c(N(CCN)C1=O)cccc2)c...,NaN,NaN,NaN,NaN,NaN,10000220,ITPLTCJNLBCHHJ-MOPGFXCFSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263908,mbIL-2 mRNA (mRNA for the membrane bound human...,NaN,L2S9NE5V6V,496142221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263909,mbIL-2 mRNA encoded fusion protein of the tran...,NaN,D7G89L3KE3,496142027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263910,miR-200c-3p | MIR-200C-3P,NaN,ZR8P5M7HVX,482554707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263911,"α-({3,5-bis[3-(triethoxysilyl)propoxy]phenyl}m...",NaN,QTG36RJ4J5,478839222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_ndc_unii_pubchem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 264782 entries, 0 to 263912
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   UNII_Name                 167726 non-null  object
 1   PRODUCTNDC                8326 non-null    object
 2   UNII                      163456 non-null  object
 3   PubChem_SID               156762 non-null  object
 4   RN                        124710 non-null  object
 5   UNII_SMILES               122162 non-null  object
 6   PubChem_Name              124656 non-null  object
 7   cmpdsynonym               123305 non-null  object
 8   InChI                     124656 non-null  object
 9   PubChem_Canonical_SMILES  124656 non-null  object
 10  iupacname                 124197 non-null  object
 11  PubChem CID               219563 non-null  object
 12  InChIKEY                  219562 non-null  object
dtypes: object(13)
memory usage: 28.3+ MB


### 3.3 Filter CAS from PubChem cmpdsynonym column

In [19]:
import re

def extract_unique_cas_numbers(synonyms):
    segments = str(synonyms).split('|')
    cas_pattern = r'^\d{2,7}-\d{2}-\d$'
    cas_numbers = [segment for segment in segments if re.match(cas_pattern, segment)]
    unique_cas_numbers = sorted(set(cas_numbers))
    return ';'.join(unique_cas_numbers)

def merge_cas(existing, extracted):
    if pd.isna(existing):
        existing_set = set()
    else:
        existing_set = set(existing.split(';'))
    
    if pd.isna(extracted):
        extracted_set = set()
    else:
        extracted_set = set(extracted.split(';'))
    
    merged = existing_set.union(extracted_set)
    # Filter out any empty strings or 'nan' from the merged set
    merged = {x for x in merged if x and x.lower() != 'nan'}
    return ';'.join(sorted(merged))

In [20]:
def clean_column(col):
    return col if col and col.strip() else pd.NA

df_filter = df_ndc_unii_pubchem.copy()
df_filter.rename(columns={'RN':'CAS'}, inplace=True)
df_filter['extracted_CAS'] = df_filter['cmpdsynonym'].apply(extract_unique_cas_numbers)
df_filter['CAS'] = df_filter.apply(lambda row: merge_cas(row['CAS'], row['extracted_CAS']), axis=1)

df_filter['CAS'] = df_filter['CAS'].apply(clean_column)
df_filter.drop(columns=['extracted_CAS'], inplace=True)
df_filter = df_filter.assign(CAS=df_filter['CAS'].str.split(';')).explode('CAS')
df_filter['CAS'] = df_filter['CAS'].str.strip()
df_filter

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,PubChem CID,InChIKEY
0,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",NaN,07OP6H4V4A,198939309,14992-62-2,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,Acetyl-DL-carnitine,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate,1,RDHQFKQIGNGIED-UHFFFAOYSA-N
0,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",NaN,07OP6H4V4A,198939309,870-77-9,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,Acetyl-DL-carnitine,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate,1,RDHQFKQIGNGIED-UHFFFAOYSA-N
1,PHENYLETHANOLAMINE,NaN,2P4Y56479O,198959170,1936-63-6,c1ccc(cc1)C(O)CN,2-Amino-1-phenylethanol,2-Amino-1-phenylethanol|phenylethanolamine|756...,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",C1=CC=C(C=C1)C(CN)O,2-amino-1-phenylethanol,1000,ULSIYEODSMZIPX-UHFFFAOYSA-N
1,PHENYLETHANOLAMINE,NaN,2P4Y56479O,198959170,7568-93-6,c1ccc(cc1)C(O)CN,2-Amino-1-phenylethanol,2-Amino-1-phenylethanol|phenylethanolamine|756...,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",C1=CC=C(C=C1)C(CN)O,2-amino-1-phenylethanol,1000,ULSIYEODSMZIPX-UHFFFAOYSA-N
2,"3-CHLORO-1,1,1-TRIFLUOROPROPANE",NaN,5C535H1OII,252164000,460-35-5,C(CCCl)(F)(F)F,NaN,NaN,NaN,NaN,NaN,10000,ZPIFKCVYZBVZIV-UHFFFAOYSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263908,mbIL-2 mRNA (mRNA for the membrane bound human...,NaN,L2S9NE5V6V,496142221,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263909,mbIL-2 mRNA encoded fusion protein of the tran...,NaN,D7G89L3KE3,496142027,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263910,miR-200c-3p | MIR-200C-3P,NaN,ZR8P5M7HVX,482554707,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263911,"α-({3,5-bis[3-(triethoxysilyl)propoxy]phenyl}m...",NaN,QTG36RJ4J5,478839222,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.4 Add DrugBank

In [21]:
df_ndc_unii_pubchem_drugbank = df_filter.copy()

DrugBand and CID

In [22]:
df_ndc_unii_pubchem_drugbank = pd.merge(df_ndc_unii_pubchem_drugbank, df_drugbank_cid, left_on='PubChem CID', right_on='PubChem_CID', how='outer')
df_ndc_unii_pubchem_drugbank = merge_column(df_ndc_unii_pubchem_drugbank, 'PubChem CID', 'PubChem_CID', 'PubChemCID')
df_ndc_unii_pubchem_drugbank

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,InChIKEY,DrugBank_ID,Drug_Name,PubChemCID
0,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",NaN,07OP6H4V4A,198939309,14992-62-2,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,Acetyl-DL-carnitine,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate,RDHQFKQIGNGIED-UHFFFAOYSA-N,NaN,NaN,1
1,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",NaN,07OP6H4V4A,198939309,870-77-9,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,Acetyl-DL-carnitine,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,3-acetyloxy-4-(trimethylazaniumyl)butanoate,RDHQFKQIGNGIED-UHFFFAOYSA-N,NaN,NaN,1
2,PHENYLETHANOLAMINE,NaN,2P4Y56479O,198959170,1936-63-6,c1ccc(cc1)C(O)CN,2-Amino-1-phenylethanol,2-Amino-1-phenylethanol|phenylethanolamine|756...,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",C1=CC=C(C=C1)C(CN)O,2-amino-1-phenylethanol,ULSIYEODSMZIPX-UHFFFAOYSA-N,NaN,NaN,1000
3,PHENYLETHANOLAMINE,NaN,2P4Y56479O,198959170,7568-93-6,c1ccc(cc1)C(O)CN,2-Amino-1-phenylethanol,2-Amino-1-phenylethanol|phenylethanolamine|756...,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",C1=CC=C(C=C1)C(CN)O,2-amino-1-phenylethanol,ULSIYEODSMZIPX-UHFFFAOYSA-N,NaN,NaN,1000
4,"3-CHLORO-1,1,1-TRIFLUOROPROPANE",NaN,5C535H1OII,252164000,460-35-5,C(CCCl)(F)(F)F,NaN,NaN,NaN,NaN,NaN,ZPIFKCVYZBVZIV-UHFFFAOYSA-N,NaN,NaN,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278711,mbIL-2 mRNA (mRNA for the membrane bound human...,NaN,L2S9NE5V6V,496142221,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278712,mbIL-2 mRNA encoded fusion protein of the tran...,NaN,D7G89L3KE3,496142027,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278713,miR-200c-3p | MIR-200C-3P,NaN,ZR8P5M7HVX,482554707,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278714,"α-({3,5-bis[3-(triethoxysilyl)propoxy]phenyl}m...",NaN,QTG36RJ4J5,478839222,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


DrugBand and SID

In [23]:
df_ndc_unii_pubchem_drugbank_v1 = pd.merge(df_ndc_unii_pubchem_drugbank, df_drugbank_SID, left_on='PubChem_SID', right_on='PubChem_SID', how='outer')
df_ndc_unii_pubchem_drugbank_v1 = merge_column(df_ndc_unii_pubchem_drugbank_v1, 'DrugBank_ID_x', 'DrugBank_ID_y', 'DrugBank_ID')
df_ndc_unii_pubchem_drugbank_v1 = merge_string_columns(df_ndc_unii_pubchem_drugbank_v1, ['Drug_Name_x', 'Drug_Name_y'], 'Drug_Name', ' | ', split_values=True)
df_ndc_unii_pubchem_drugbank_v1.replace('', pd.NA, inplace=True)
df_ndc_unii_pubchem_drugbank_v1

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,InChIKEY,PubChemCID,DrugBank_ID,Drug_Name
0,DIHYDRO-.ALPHA.-ERGOCRYPTINE,NaN,202229IR8Y,175265280,25447-66-9,N12[C@@](O)(O[C@](C(C)C)(C1=O)NC([C@@H]3C[C@@]...,Dihydroergocryptine,Dihydroergocryptine|Dihydro-alpha-ergocryptine...,InChI=1S/C32H43N5O5/c1-17(2)12-25-29(39)36-11-...,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]3(N1C(=O)[C@](...,"(6aR,9R,10aR)-N-[(1S,2S,4R,7S)-2-hydroxy-7-(2-...",PBUNVLRHZGSROC-VTIMJTGVSA-N,114948,NaN,<NA>
1,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,139404-48-1,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-SEINRUQRSA-M,11431811,NaN,<NA>
2,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,411207-31-3,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-SEINRUQRSA-M,11431811,NaN,<NA>
3,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,139404-48-1,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-KFEMZTBUSA-M,11431811,NaN,<NA>
4,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,411207-31-3,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-KFEMZTBUSA-M,11431811,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280479,ZIZANIA LATIFOLIA STEM,NaN,8VY77GE7VB,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
280480,ZOLACABTAGENE AUTOLEUCEL,NaN,DU963TX85S,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
280481,ZOLBETUXIMAB,0469-3425,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
280482,ZUNIBERGENE ROCPARVOVEC,NaN,VFF5C4QV72,NaN,2856381-73-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


Remaining DrugBank ID

In [24]:
df_ndc_unii_pubchem_drugbank_v2 = pd.concat([df_ndc_unii_pubchem_drugbank_v1, df_drugbank_other], ignore_index=True)
df_ndc_unii_pubchem_drugbank_v2.drop(columns = ['CAS_Number','PubChem_CID'], inplace=True)
df_ndc_unii_pubchem_drugbank_v2

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,InChIKEY,PubChemCID,DrugBank_ID,Drug_Name
0,DIHYDRO-.ALPHA.-ERGOCRYPTINE,NaN,202229IR8Y,175265280,25447-66-9,N12[C@@](O)(O[C@](C(C)C)(C1=O)NC([C@@H]3C[C@@]...,Dihydroergocryptine,Dihydroergocryptine|Dihydro-alpha-ergocryptine...,InChI=1S/C32H43N5O5/c1-17(2)12-25-29(39)36-11-...,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]3(N1C(=O)[C@](...,"(6aR,9R,10aR)-N-[(1S,2S,4R,7S)-2-hydroxy-7-(2-...",PBUNVLRHZGSROC-VTIMJTGVSA-N,114948,NaN,<NA>
1,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,139404-48-1,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-SEINRUQRSA-M,11431811,NaN,<NA>
2,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,411207-31-3,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-SEINRUQRSA-M,11431811,NaN,<NA>
3,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,139404-48-1,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-KFEMZTBUSA-M,11431811,NaN,<NA>
4,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,411207-31-3,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-KFEMZTBUSA-M,11431811,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19452,Exidavnemab
287418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19453,Imciromab pentetate
287419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19454,Cetyl oleate
287420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19455,Cetyl myristoleate


### 3.5 Filter DrugBank ID from PubChem cmpdsynonym column

In [25]:
def extract_unique_drugbank_ids(synonyms):
    segments = str(synonyms).split('|')
    drugbank_pattern = r'^DB\d{5}$'
    drugbank_ids = [segment for segment in segments if re.match(drugbank_pattern, segment)]
    unique_drugbank_ids = sorted(set(drugbank_ids))
    return ';'.join(unique_drugbank_ids)

def merge_drugbank_ids(existing, extracted):
    if pd.isna(existing):
        return extracted
    if pd.isna(extracted):
        return existing
    existing_set = set(existing.split(';')) if existing else set()
    extracted_set = set(extracted.split(';')) if extracted else set()
    merged = existing_set.union(extracted_set)
    # Filter out any empty strings or 'nan' from the merged set
    merged = {x for x in merged if x and x.lower() != 'nan'}
    return ';'.join(sorted(merged))

def clean_column(col):
    return col if col and col.strip() else pd.NA

df_filter = df_ndc_unii_pubchem_drugbank_v2.copy()
df_filter['extracted_DB'] = df_filter['cmpdsynonym'].apply(extract_unique_drugbank_ids)
df_filter['DrugBank_ID'] = df_filter.apply(lambda row: merge_drugbank_ids(row['DrugBank_ID'], row['extracted_DB']), axis=1)

df_filter['DrugBank_ID'] = df_filter['DrugBank_ID'].apply(clean_column)
df_filter.drop(columns=['extracted_DB'], inplace=True)

df_expanded = df_filter.assign(DrugBank_ID=df_filter['DrugBank_ID'].str.split(';')).explode('DrugBank_ID')
df_expanded['DrugBank_ID'] = df_expanded['DrugBank_ID'].str.strip()
df_expanded.drop_duplicates(inplace=True)
df_expanded

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,InChIKEY,PubChemCID,DrugBank_ID,Drug_Name
0,DIHYDRO-.ALPHA.-ERGOCRYPTINE,NaN,202229IR8Y,175265280,25447-66-9,N12[C@@](O)(O[C@](C(C)C)(C1=O)NC([C@@H]3C[C@@]...,Dihydroergocryptine,Dihydroergocryptine|Dihydro-alpha-ergocryptine...,InChI=1S/C32H43N5O5/c1-17(2)12-25-29(39)36-11-...,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]3(N1C(=O)[C@](...,"(6aR,9R,10aR)-N-[(1S,2S,4R,7S)-2-hydroxy-7-(2-...",PBUNVLRHZGSROC-VTIMJTGVSA-N,114948,DB11274,<NA>
1,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,139404-48-1,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-SEINRUQRSA-M,11431811,<NA>,<NA>
2,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,411207-31-3,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-SEINRUQRSA-M,11431811,<NA>,<NA>
3,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,139404-48-1,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-KFEMZTBUSA-M,11431811,<NA>,<NA>
4,TIOTROPIUM BROMIDE MONOHYDRATE,0597-0075;50090-4248;68180-964,L64SXO195N,175265281,411207-31-3,[C@@]12([C@H]3C[C@@H](OC(C(c4sccc4)(c5sccc5)O)...,Tiotropium bromide monohydrate,Tiotropium bromide hydrate|139404-48-1|Tiotrop...,InChI=1S/C19H22NO4S2.BrH.H2O/c1-20(2)12-9-11(1...,C[N+]1([C@@H]2CC(C[C@H]1[C@H]3[C@@H]2O3)OC(=O)...,"[(1R,2R,4S,5S)-9,9-dimethyl-3-oxa-9-azoniatric...",MQLXPRBEAHBZTK-KFEMZTBUSA-M,11431811,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19452,Exidavnemab
287418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19453,Imciromab pentetate
287419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19454,Cetyl oleate
287420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19455,Cetyl myristoleate


### 3.6 Final Data Cleaning

In [26]:
df_data = df_expanded.copy()

In [28]:
def data_cleaning(df, column):
    df_no_duplicates = df[~(df[column].duplicated(keep=False) & df[column].notna()) | df[column].isna()]
    df_duplicates = df[df[column].notna() & df.duplicated(column, keep=False)].sort_values(column)

    def choose_separator(colname):
        if 'name' in colname.lower():
            return ' | '
        else:
            return ';'

    def merge_entries(series):
        filtered = series.dropna().astype(str)
        colname = series.name
        sep = choose_separator(colname)
        return sep.join(filtered.unique())

    df_duplicates_merged = df_duplicates.groupby(column).agg(merge_entries).reset_index()

    return df_no_duplicates, df_duplicates_merged

Remove Duplication based on PubChem CID

In [29]:
# Apply the data_cleaning function to the 'PubChem CID' column
df_data['PubChemCID'] = df_data['PubChemCID'].str.split(';')
df_data = df_data.explode('PubChemCID')
df_data_CID_noduplications, df_data_CID_duplications = data_cleaning(df_data, 'PubChemCID')

final_drugs = pd.concat([df_data_CID_noduplications, df_data_CID_duplications], ignore_index=True)
final_drugs.replace('', pd.NA, inplace=True)
final_drugs.drop_duplicates(inplace=True)
final_drugs.reset_index(drop=True, inplace=True)
final_drugs

,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,InChIKEY,PubChemCID,DrugBank_ID,Drug_Name
0,DIHYDRO-.ALPHA.-ERGOCRYPTINE,NaN,202229IR8Y,175265280,25447-66-9,N12[C@@](O)(O[C@](C(C)C)(C1=O)NC([C@@H]3C[C@@]...,Dihydroergocryptine,Dihydroergocryptine|Dihydro-alpha-ergocryptine...,InChI=1S/C32H43N5O5/c1-17(2)12-25-29(39)36-11-...,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]3(N1C(=O)[C@](...,"(6aR,9R,10aR)-N-[(1S,2S,4R,7S)-2-hydroxy-7-(2-...",PBUNVLRHZGSROC-VTIMJTGVSA-N,114948,DB11274,<NA>
1,NORETHINDRONE,0378-7272;0378-7292;0480-3475;50090-6161;50102...,T18F433X4S,175265282,68-22-4,[C@]12([C@]([C@@]3(CCC=4[C@@]([C@]3(CC1)[H])(C...,Norethindrone,norethindrone|Norethisterone|68-22-4|Norethist...,InChI=1S/C20H26O2/c1-3-20(22)11-9-18-17-6-4-13...,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C#C)O)CCC4...,"(8R,9S,10R,13S,14S,17R)-17-ethynyl-17-hydroxy-...",VIKNJXKGJWUCNN-XGXHKTLJSA-N,6230,DB00717,Norethisterone
2,ETHYL LAUROYL ARGINATE HYDROCHLORIDE,NaN,XPD6ZY79TB,175265283,60372-77-2,Cl.CCOC(=O)[C@H](CCCNC(=N)N)NC(=O)CCCCCCCCCCC,NaN,NaN,NaN,NaN,NaN,CUBZMGWLVMQKNE-LMOVPXPDSA-N,25229630,<NA>,<NA>
3,SODIUM LAUROYL ISETHIONATE,NaN,M590021Z02,175265284,7381-01-3,O(CCS([O-])(=O)=O)C(CCCCCCCCCCC)=O.[Na+],NaN,NaN,NaN,NaN,NaN,BRMSVEGRHOZCAM-UHFFFAOYSA-M,23668826,<NA>,<NA>
4,PYRIDOXINE TRIPALMITATE,NaN,KXS58JW4OT,175265286,4372-46-7,c1(c(OC(=O)CCCCCCCCCCCCCCC)c(C)ncc1COC(=O)CCCC...,NaN,NaN,NaN,NaN,NaN,UDRYFKCHZFVZGJ-UHFFFAOYSA-N,20390,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274775,PHENYLACETIC ACID,<NA>,ER5I1W795A,175267987,17303-65-0;103-82-2;51146-16-8,c1ccc(cc1)CC(=O)O,Phenylacetic Acid,PHENYLACETIC ACID|2-Phenylacetic acid|Benzenea...,"InChI=1S/C8H8O2/c9-8(10)6-7-4-2-1-3-5-7/h1-5H,...",C1=CC=C(C=C1)CC(=O)O,2-phenylacetic acid,WLJVXDMOQOGPHL-UHFFFAOYSA-N,999,DB09269,Phenylacetic acid
274776,"METAZOCINE, CIS-(+/-)- | METAZOCINE, CIS-(±)- ...",<NA>,2AO0073XHO;670ER53976,198985269;198979357,25144-79-0;67009-58-9,C[C@@]12c3c(C[C@@H]([C@H]1C)N(CC2)C)ccc(c3)O;c...,<NA>,<NA>,<NA>,<NA>,<NA>,YGSVZRIZCHZUHB-ONERCXAPSA-N,9991509,<NA>,<NA>
274777,"KHELLACTONE, TRANS-(-)- | KHELLACTONE, TRANS-(...",<NA>,CIJ6HK1AGB;17SSV9BI4T,377308985;377308510,23458-04-0;15575-68-5,O[C@H]1c2c3c(ccc2OC([C@@H]1O)(C)C)ccc(o3)=O,<NA>,<NA>,<NA>,<NA>,<NA>,HKXQUNNSKMWIKJ-WCQYABFASA-N,9992853,<NA>,<NA>
274778,"CLAVICIPITIC ACID, TRANS-(-)- | CLAVICIPITIC A...",<NA>,34R1382X09;374VHV303R,346539459;318691391,33062-26-9;84986-03-8,C([C@H]1N[C@H](C(=O)O)Cc2c[nH]c3cccc1c32)=C(C)C,<NA>,<NA>,<NA>,<NA>,<NA>,VZMAHZAQMKNJIG-KGLIPLIRSA-N,9993281,<NA>,<NA>


Remove Duplication based on CAS

In [30]:
# for the rows that have empty PubChem CID, merge the row which has the same CAS number
cid_empty = final_drugs[final_drugs['PubChemCID'].isna()]
cid_exist = final_drugs[final_drugs['PubChemCID'].notna()]

cid_empty['CAS'] = cid_empty['CAS'].str.split(';')
cid_empty = cid_empty.explode('CAS')
cid_empty['CAS'] = cid_empty['CAS'].str.strip()
cid_empty_no_duplicates, cid_empty_duplicates = data_cleaning(cid_empty, 'CAS')

final_drugs = pd.concat([cid_exist, cid_empty_duplicates, cid_empty_no_duplicates], ignore_index=True)
final_drugs.replace('', pd.NA, inplace=True)
final_drugs.drop_duplicates(inplace=True)
final_drugs.reset_index(drop=True, inplace=True)
final_drugs


C:\Users\shunn\AppData\Local\Temp\ipykernel_29836\3669193150.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cid_empty['CAS'] = cid_empty['CAS'].str.split(';')


,UNII_Name,PRODUCTNDC,UNII,PubChem_SID,CAS,UNII_SMILES,PubChem_Name,cmpdsynonym,InChI,PubChem_Canonical_SMILES,iupacname,InChIKEY,PubChemCID,DrugBank_ID,Drug_Name
0,DIHYDRO-.ALPHA.-ERGOCRYPTINE,NaN,202229IR8Y,175265280,25447-66-9,N12[C@@](O)(O[C@](C(C)C)(C1=O)NC([C@@H]3C[C@@]...,Dihydroergocryptine,Dihydroergocryptine|Dihydro-alpha-ergocryptine...,InChI=1S/C32H43N5O5/c1-17(2)12-25-29(39)36-11-...,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]3(N1C(=O)[C@](...,"(6aR,9R,10aR)-N-[(1S,2S,4R,7S)-2-hydroxy-7-(2-...",PBUNVLRHZGSROC-VTIMJTGVSA-N,114948,DB11274,<NA>
1,NORETHINDRONE,0378-7272;0378-7292;0480-3475;50090-6161;50102...,T18F433X4S,175265282,68-22-4,[C@]12([C@]([C@@]3(CCC=4[C@@]([C@]3(CC1)[H])(C...,Norethindrone,norethindrone|Norethisterone|68-22-4|Norethist...,InChI=1S/C20H26O2/c1-3-20(22)11-9-18-17-6-4-13...,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C#C)O)CCC4...,"(8R,9S,10R,13S,14S,17R)-17-ethynyl-17-hydroxy-...",VIKNJXKGJWUCNN-XGXHKTLJSA-N,6230,DB00717,Norethisterone
2,ETHYL LAUROYL ARGINATE HYDROCHLORIDE,NaN,XPD6ZY79TB,175265283,60372-77-2,Cl.CCOC(=O)[C@H](CCCNC(=N)N)NC(=O)CCCCCCCCCCC,NaN,NaN,NaN,NaN,NaN,CUBZMGWLVMQKNE-LMOVPXPDSA-N,25229630,<NA>,<NA>
3,SODIUM LAUROYL ISETHIONATE,NaN,M590021Z02,175265284,7381-01-3,O(CCS([O-])(=O)=O)C(CCCCCCCCCCC)=O.[Na+],NaN,NaN,NaN,NaN,NaN,BRMSVEGRHOZCAM-UHFFFAOYSA-M,23668826,<NA>,<NA>
4,PYRIDOXINE TRIPALMITATE,NaN,KXS58JW4OT,175265286,4372-46-7,c1(c(OC(=O)CCCCCCCCCCCCCCC)c(C)ncc1COC(=O)CCCC...,NaN,NaN,NaN,NaN,NaN,UDRYFKCHZFVZGJ-UHFFFAOYSA-N,20390,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19452,Exidavnemab
273382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19453,Imciromab pentetate
273383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19454,Cetyl oleate
273384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19455,Cetyl myristoleate


In [31]:
final_drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273386 entries, 0 to 273385
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   UNII_Name                 164274 non-null  object
 1   PRODUCTNDC                8236 non-null    object
 2   UNII                      160004 non-null  object
 3   PubChem_SID               155131 non-null  object
 4   CAS                       137038 non-null  object
 5   UNII_SMILES               120138 non-null  object
 6   PubChem_Name              123357 non-null  object
 7   cmpdsynonym               122006 non-null  object
 8   InChI                     123357 non-null  object
 9   PubChem_Canonical_SMILES  123357 non-null  object
 10  iupacname                 122922 non-null  object
 11  InChIKEY                  217529 non-null  object
 12  PubChemCID                220855 non-null  object
 13  DrugBank_ID               19500 non-null   object
 14  Drug

## 4. BioMedgraphica ID

In [32]:
biomedgraphica_drug = final_drugs.copy()
biomedgraphica_drug = biomedgraphica_drug.rename(columns={'PRODUCTNDC':'NDC', 'CAS':'CAS_RN', 
                                                          'iupacname':'IUPAC_Name', 'Drug_Name':'DrugBank_Name', 
                                                          'PubChemCID':'PubChem_CID', 'cmpdsynonym':'PubChem_Synonym'})
biomedgraphica_drug = biomedgraphica_drug.sort_values(by=['PubChem_CID', 'PubChem_SID', 'CAS_RN', 'UNII', 'DrugBank_ID'], na_position='last')
biomedgraphica_drug.reset_index(drop=True, inplace=True)

max_length = len(str(len(biomedgraphica_drug)))
biomedgraphica_drug['BioMedGraphica_ID'] = ['BMG_DG' + str(i).zfill(max_length) for i in range(1, len(biomedgraphica_drug) + 1)]
column_order = ['BioMedGraphica_ID', 'PubChem_CID', 'PubChem_SID', 'PubChem_Name', 
                'CAS_RN', 'IUPAC_Name', 'UNII', 'UNII_Name', 'NDC', 'DrugBank_ID', 'DrugBank_Name', 
                'PubChem_Canonical_SMILES', 'UNII_SMILES', 'InChI', 'InChIKEY', 'PubChem_Synonym']
biomedgraphica_drug = biomedgraphica_drug[column_order]
biomedgraphica_drug.replace('', pd.NA, inplace=True)
biomedgraphica_drug

,BioMedGraphica_ID,PubChem_CID,PubChem_SID,PubChem_Name,CAS_RN,IUPAC_Name,UNII,UNII_Name,NDC,DrugBank_ID,DrugBank_Name,PubChem_Canonical_SMILES,UNII_SMILES,InChI,InChIKEY,PubChem_Synonym
0,BMG_DG000001,1,198939309,Acetyl-DL-carnitine,14992-62-2;870-77-9,3-acetyloxy-4-(trimethylazaniumyl)butanoate,07OP6H4V4A,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",<NA>,<NA>,<NA>,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,RDHQFKQIGNGIED-UHFFFAOYSA-N,Acetyl-DL-carnitine|acetylcarnitine|14992-62-2...
1,BMG_DG000002,1000,198959170,2-Amino-1-phenylethanol,1936-63-6;7568-93-6,2-amino-1-phenylethanol,2P4Y56479O,PHENYLETHANOLAMINE,<NA>,<NA>,<NA>,C1=CC=C(C=C1)C(CN)O,c1ccc(cc1)C(O)CN,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",ULSIYEODSMZIPX-UHFFFAOYSA-N,2-Amino-1-phenylethanol|phenylethanolamine|756...
2,BMG_DG000003,10000,252164000,NaN,460-35-5,NaN,5C535H1OII,"3-CHLORO-1,1,1-TRIFLUOROPROPANE",NaN,<NA>,<NA>,NaN,C(CCCl)(F)(F)F,NaN,ZPIFKCVYZBVZIV-UHFFFAOYSA-N,NaN
3,BMG_DG000004,100001,NaN,N-Benzoylhistidine,14056-33-8,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid,NaN,NaN,NaN,<NA>,<NA>,C1=CC=C(C=C1)C(=O)NC(CC2=CN=CN2)C(=O)O,NaN,InChI=1S/C13H13N3O3/c17-12(9-4-2-1-3-5-9)16-11...,AUDPUFBIVWMAED-UHFFFAOYSA-N,N-Benzoylhistidine|benzoylhistidine|3-(1H-Imid...
4,BMG_DG000005,10000220,474494223,NaN,115973-28-9,NaN,EFG5T3366P,"N,N-DIDESMETHYLDILTIAZEM | N,N-Didesmethyldilt...",NaN,<NA>,<NA>,NaN,O(C(C)=O)[C@@H]1[C@@H](Sc2c(N(CCN)C1=O)cccc2)c...,NaN,ITPLTCJNLBCHHJ-MOPGFXCFSA-N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273381,BMG_DG273382,NaN,NaN,NaN,<NA>,NaN,NaN,ZINC CHLORIDE,0409-4090;37662-4087;37662-4088;37662-4089;376...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
273382,BMG_DG273383,NaN,NaN,NaN,<NA>,NaN,NaN,ZINC OXIDE,0132-0194;0132-0323;0132-0324;0132-0333;0132-0...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
273383,BMG_DG273384,NaN,NaN,NaN,<NA>,NaN,NaN,ZINC SULFATE,0517-6101;0517-6103;0517-8005;65219-401;65219-...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
273384,BMG_DG273385,NaN,NaN,NaN,<NA>,NaN,NaN,ZINGIBER OFFICINALE (GINGER) ROOT EXTRACT,74458-115;84778-017,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN


In [33]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug.csv'
biomedgraphica_drug.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug.csv


## 5. File Generation

In [5]:
import pandas as pd
import os
from pathlib import Path

current_working_dir = Path(os.getcwd()).resolve()
grandparent_dir = current_working_dir.parent.parent.parent
target_dir = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug' / 'BioMedGraphica_Drug.csv'
biomedgraphica_drug = pd.read_csv(target_dir, dtype=str)

### 5.1 BioChem

In [35]:
biochem = biomedgraphica_drug[['BioMedGraphica_ID', 'PubChem_Canonical_SMILES', 'UNII_SMILES', 'InChI', 'InChIKEY']]
biochem

,BioMedGraphica_ID,PubChem_Canonical_SMILES,UNII_SMILES,InChI,InChIKEY
0,BMG_DG000001,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,C(C([O-])=O)C(OC(=O)C)C[N+](C)(C)C,InChI=1S/C9H17NO4/c1-7(11)14-8(5-9(12)13)6-10(...,RDHQFKQIGNGIED-UHFFFAOYSA-N
1,BMG_DG000002,C1=CC=C(C=C1)C(CN)O,c1ccc(cc1)C(O)CN,"InChI=1S/C8H11NO/c9-6-8(10)7-4-2-1-3-5-7/h1-5,...",ULSIYEODSMZIPX-UHFFFAOYSA-N
2,BMG_DG000003,NaN,C(CCCl)(F)(F)F,NaN,ZPIFKCVYZBVZIV-UHFFFAOYSA-N
3,BMG_DG000004,C1=CC=C(C=C1)C(=O)NC(CC2=CN=CN2)C(=O)O,NaN,InChI=1S/C13H13N3O3/c17-12(9-4-2-1-3-5-9)16-11...,AUDPUFBIVWMAED-UHFFFAOYSA-N
4,BMG_DG000005,NaN,O(C(C)=O)[C@@H]1[C@@H](Sc2c(N(CCN)C1=O)cccc2)c...,NaN,ITPLTCJNLBCHHJ-MOPGFXCFSA-N
...,...,...,...,...,...
273381,BMG_DG273382,NaN,NaN,NaN,NaN
273382,BMG_DG273383,NaN,NaN,NaN,NaN
273383,BMG_DG273384,NaN,NaN,NaN,NaN
273384,BMG_DG273385,NaN,NaN,NaN,NaN


In [36]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_BioChem.csv'
biochem.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_BioChem.csv


### 5.2 Name and ID

GUI Name

In [37]:
def merge_string_columns(df, columns, merge_name, separator=' | '):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                combined.update(row[column].split(separator))
        return separator.join(sorted(combined))

    # Apply the function to each row and create a new column
    combined_column_name = merge_name
    df[combined_column_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

gui_name = biomedgraphica_drug.copy()
gui_name = merge_string_columns(gui_name, ['PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name'], 'Drug_Name_List')
gui_name = gui_name[['BioMedGraphica_ID', 'Drug_Name_List']]
gui_name

,BioMedGraphica_ID,Drug_Name_List
0,BMG_DG000001,3-acetyloxy-4-(trimethylazaniumyl)butanoate | ...
1,BMG_DG000002,2-Amino-1-phenylethanol | 2-amino-1-phenyletha...
2,BMG_DG000003,"3-CHLORO-1,1,1-TRIFLUOROPROPANE"
3,BMG_DG000004,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid...
4,BMG_DG000005,"N,N-DIDESMETHYLDILTIAZEM | N,N-Didesmethyldilt..."
...,...,...
273381,BMG_DG273382,ZINC CHLORIDE
273382,BMG_DG273383,ZINC OXIDE
273383,BMG_DG273384,ZINC SULFATE
273384,BMG_DG273385,ZINGIBER OFFICINALE (GINGER) ROOT EXTRACT


In [38]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_GUI_Name.csv'
gui_name.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_GUI_Name.csv


LLM Name and ID

In [39]:
llm_name_id = biomedgraphica_drug.copy()

llm_name_id['PubChem_CID'] = llm_name_id['PubChem_CID'].apply(
    lambda x: ' | '.join(f"PubChem CID:{id}" for id in x.split(';')) if pd.notna(x) and x != '' else x
)
llm_name_id['PubChem_SID'] = llm_name_id['PubChem_SID'].apply(
    lambda x: ' | '.join(f"PubChem SID:{id}" for id in x.split(';')) if pd.notna(x) and x != '' else x
)
llm_name_id['CAS_RN'] = llm_name_id['CAS_RN'].apply(
    lambda x: ' | '.join(f"CAS RN:{id}" for id in x.split(';')) if pd.notna(x) and x != '' else x
)
llm_name_id['UNII'] = llm_name_id['UNII'].apply(
    lambda x: ' | '.join(f"UNII:{id}" for id in x.split(';')) if pd.notna(x) and x != '' else x
)
llm_name_id['NDC'] = llm_name_id['NDC'].apply(
    lambda x: ' | '.join(f"NDC:{id}" for id in x.split(';')) if pd.notna(x) and x != '' else x
)
llm_name_id['DrugBank_ID'] = llm_name_id['DrugBank_ID'].apply(
    lambda x: ' | '.join(f"DrugBank ID:{id}" for id in x.split(';')) if pd.notna(x) and x != '' else x
)

llm_name_id.drop(columns=['PubChem_Canonical_SMILES', 'UNII_SMILES', 'InChI', 'InChIKEY', 'PubChem_Synonym'], inplace=True)
column_order = ['BioMedGraphica_ID', 'PubChem_Name', 'PubChem_CID', 'PubChem_SID', 
                'UNII_Name', 'UNII', 'DrugBank_Name', 'DrugBank_ID',
                'IUPAC_Name', 'CAS_RN', 'NDC']
llm_name_id = llm_name_id[column_order]
llm_name_id

,BioMedGraphica_ID,PubChem_Name,PubChem_CID,PubChem_SID,UNII_Name,UNII,DrugBank_Name,DrugBank_ID,IUPAC_Name,CAS_RN,NDC
0,BMG_DG000001,Acetyl-DL-carnitine,PubChem CID:1,PubChem SID:198939309,"ACETYLCARNITINE, (±)- | ACETYLCARNITINE, (+/-)-",UNII:07OP6H4V4A,<NA>,<NA>,3-acetyloxy-4-(trimethylazaniumyl)butanoate,CAS RN:14992-62-2 | CAS RN:870-77-9,<NA>
1,BMG_DG000002,2-Amino-1-phenylethanol,PubChem CID:1000,PubChem SID:198959170,PHENYLETHANOLAMINE,UNII:2P4Y56479O,<NA>,<NA>,2-amino-1-phenylethanol,CAS RN:1936-63-6 | CAS RN:7568-93-6,<NA>
2,BMG_DG000003,NaN,PubChem CID:10000,PubChem SID:252164000,"3-CHLORO-1,1,1-TRIFLUOROPROPANE",UNII:5C535H1OII,<NA>,<NA>,NaN,CAS RN:460-35-5,NaN
3,BMG_DG000004,N-Benzoylhistidine,PubChem CID:100001,NaN,NaN,NaN,<NA>,<NA>,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid,CAS RN:14056-33-8,NaN
4,BMG_DG000005,NaN,PubChem CID:10000220,PubChem SID:474494223,"N,N-DIDESMETHYLDILTIAZEM | N,N-Didesmethyldilt...",UNII:EFG5T3366P,<NA>,<NA>,NaN,CAS RN:115973-28-9,NaN
...,...,...,...,...,...,...,...,...,...,...,...
273381,BMG_DG273382,NaN,NaN,NaN,ZINC CHLORIDE,NaN,<NA>,<NA>,NaN,<NA>,NDC:0409-4090 | NDC:37662-4087 | NDC:37662-408...
273382,BMG_DG273383,NaN,NaN,NaN,ZINC OXIDE,NaN,<NA>,<NA>,NaN,<NA>,NDC:0132-0194 | NDC:0132-0323 | NDC:0132-0324 ...
273383,BMG_DG273384,NaN,NaN,NaN,ZINC SULFATE,NaN,<NA>,<NA>,NaN,<NA>,NDC:0517-6101 | NDC:0517-6103 | NDC:0517-8005 ...
273384,BMG_DG273385,NaN,NaN,NaN,ZINGIBER OFFICINALE (GINGER) ROOT EXTRACT,NaN,<NA>,<NA>,NaN,<NA>,NDC:74458-115 | NDC:84778-017


In [40]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_LLM_Name_ID.csv'
llm_name_id.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_LLM_Name_ID.csv


LLM Name and ID Combined

In [41]:
llm_combined = llm_name_id.copy()

def merge_string_columns(df, columns, merge_name, separator=' | '):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                combined.update(row[column].split(separator))
        return separator.join(combined)
    
    # Apply the function to each row and create a new column
    combined_column_name = merge_name
    df[combined_column_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

llm_combined = merge_string_columns(llm_combined, ['PubChem_Name', 'PubChem_CID', 'PubChem_SID', 
                'UNII_Name', 'UNII', 'DrugBank_Name', 'DrugBank_ID',
                'IUPAC_Name', 'CAS_RN', 'NDC'], 'Names_and_IDs')
llm_combined


,BioMedGraphica_ID,Names_and_IDs
0,BMG_DG000001,"ACETYLCARNITINE, (+/-)- | UNII:07OP6H4V4A | Pu..."
1,BMG_DG000002,UNII:2P4Y56479O | 2-Amino-1-phenylethanol | CA...
2,BMG_DG000003,PubChem CID:10000 | CAS RN:460-35-5 | UNII:5C5...
3,BMG_DG000004,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid...
4,BMG_DG000005,PubChem SID:474494223 | CAS RN:115973-28-9 | N...
...,...,...
273381,BMG_DG273382,NDC:37662-4087 | NDC:37662-4095 | NDC:0409-409...
273382,BMG_DG273383,NDC:82746-205 | NDC:79950-014 | NDC:36800-260 ...
273383,BMG_DG273384,NDC:0517-6101 | NDC:0517-6103 | NDC:70771-1850...
273384,BMG_DG273385,NDC:74458-115 | NDC:84778-017 | ZINGIBER OFFIC...


In [42]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_LLM_Name_ID_Combined.csv'
llm_combined.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_LLM_Name_ID_Combined.csv


Display Name

In [43]:
display_name = biomedgraphica_drug.copy()

display_name['BMG_Drug_Name'] = display_name['IUPAC_Name'].fillna(display_name['PubChem_Name']).fillna(display_name['UNII_Name']).fillna(display_name['DrugBank_Name'])
display_name = display_name[['BioMedGraphica_ID', 'BMG_Drug_Name']]
display_name

,BioMedGraphica_ID,BMG_Drug_Name
0,BMG_DG000001,3-acetyloxy-4-(trimethylazaniumyl)butanoate
1,BMG_DG000002,2-amino-1-phenylethanol
2,BMG_DG000003,"3-CHLORO-1,1,1-TRIFLUOROPROPANE"
3,BMG_DG000004,2-benzamido-3-(1H-imidazol-5-yl)propanoic acid
4,BMG_DG000005,"N,N-DIDESMETHYLDILTIAZEM | N,N-Didesmethyldilt..."
...,...,...
273381,BMG_DG273382,ZINC CHLORIDE
273382,BMG_DG273383,ZINC OXIDE
273383,BMG_DG273384,ZINC SULFATE
273384,BMG_DG273385,ZINGIBER OFFICINALE (GINGER) ROOT EXTRACT


In [44]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_Display_Name.csv'
display_name.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_Display_Name.csv


## 6. Discription

In [1]:
import pandas as pd
import os
from pathlib import Path

current_working_dir = Path(os.getcwd()).resolve()
grandparent_dir = current_working_dir.parent.parent.parent
target_dir = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug' / 'BioMedGraphica_Drug.csv'
biomedgraphica_drug = pd.read_csv(target_dir, dtype=str)

### 6.1 From DrugBank

In [ ]:
import xml.etree.ElementTree as ET
import csv

input_file = 'full database.xml'
output_file = 'drugbank_description_data.csv'

tree = ET.parse(input_file)
root = tree.getroot()

namespace = {'ns': 'http://www.drugbank.ca'}

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    writer.writerow(['DrugBank ID', 'Description'])
    
    # go through all <drug> elements
    for drug in root.findall('ns:drug', namespace):
        # drugbank-id（primary="true"）
        primary_id = drug.find('ns:drugbank-id[@primary="true"]', namespace)
        drugbank_id = primary_id.text if primary_id is not None else 'N/A'
        
        # description
        description = drug.find('ns:description', namespace)
        if description is not None and description.text:
            description_text = description.text.replace('\n', ' ').replace('\r', ' ').strip()
        else:
            description_text = 'N/A'
        
        writer.writerow([drugbank_id, description_text])

print(f"Data has been saved to: {output_file}")

In [45]:
db_description = pd.read_csv('drugbank_description_data.csv')
db_description.dropna(subset=['Description'], inplace=True)
db_description.reset_index(drop=True, inplace=True)
db_description

,DrugBank ID,Description
0,DB00001,Lepirudin is a recombinant hirudin formed by 6...
1,DB00002,Cetuximab is a recombinant chimeric human/mous...
2,DB00003,Dornase alfa is a biosynthetic form of human d...
3,DB00004,Denileukin diftitox is an IL2-receptor-directe...
4,DB00005,Dimeric fusion protein consisting of the extra...
...,...,...
10020,DB19446,Tulmimetostat is under investigation in clinic...
10021,DB19447,Raludotatug deruxtecan is under investigation ...
10022,DB19450,TDI-01 is a small molecule is being developed ...
10023,DB19451,Ibuzatrelvir is under investigation in clinica...


In [46]:
bmg_db = biomedgraphica_drug[['BioMedGraphica_ID', 'DrugBank_ID']]
bmg_db['DrugBank_ID'] = bmg_db['DrugBank_ID'].str.split(';')
bmg_db = bmg_db.explode('DrugBank_ID')

drug_description_drugbank = pd.merge(bmg_db, db_description, left_on='DrugBank_ID', right_on='DrugBank ID', how='left')
drug_description_drugbank.drop(columns=['DrugBank ID', 'DrugBank_ID'], inplace=True)
drug_description_drugbank.rename(columns={'Description':'DrugBank'}, inplace=True)
drug_description_drugbank = drug_description_drugbank.groupby('BioMedGraphica_ID').agg({
    'DrugBank': lambda x: ';'.join(x.dropna().unique())
}).reset_index()
drug_description_drugbank.replace('', pd.NA, inplace=True)
drug_description_drugbank

C:\Users\shunn\AppData\Local\Temp\ipykernel_29836\2125587934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmg_db['DrugBank_ID'] = bmg_db['DrugBank_ID'].str.split(';')


,BioMedGraphica_ID,DrugBank
0,BMG_DG000001,<NA>
1,BMG_DG000002,<NA>
2,BMG_DG000003,<NA>
3,BMG_DG000004,<NA>
4,BMG_DG000005,<NA>
...,...,...
273381,BMG_DG273382,<NA>
273382,BMG_DG273383,<NA>
273383,BMG_DG273384,<NA>
273384,BMG_DG273385,<NA>


### 6.2 From PubChem

In [26]:
# https://pubchem.ncbi.nlm.nih.gov/docs/pug-view
# https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/1/XML?heading=Names%20and%20Identifiers 

# make a pubchem CID list
cid = biomedgraphica_drug[['PubChem_CID']].dropna(subset=['PubChem_CID'])
cid['PubChem_CID'] = cid['PubChem_CID'].str.split(';')
cid = cid.explode('PubChem_CID')
cid = cid.drop_duplicates(subset=['PubChem_CID'])
cid.rename(columns={'PubChem_CID':'CID'}, inplace=True)
# save as csv
cid.to_csv('input_cid.csv', index=False)

In [7]:
import requests
import pandas as pd
import time
import json
import os
from tqdm import tqdm

LOG_FILE = "processed.log"

def fetch_record_description(cid):
    """
    Fetch the Record Description section from PubChem for a given CID.
    
    Args:
        cid (str): The PubChem CID.
    
    Returns:
        list: A list of dictionaries, each containing the CID, String, and Reference SourceName.
    """
    base_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON"
    response = requests.get(base_url)
    
    if response.status_code != 200:
        return [{"CID": cid, "String": None, "Reference": f"HTTP {response.status_code}"}]
    
    try:
        data = response.json()
        records = []
        
        # Extract references mapping: ReferenceNumber -> SourceName
        references = {ref.get("ReferenceNumber"): ref.get("SourceName", "No Reference")
                      for ref in data.get("Record", {}).get("Reference", [])}
        
        # Traverse to "Record Description" section
        for section in data.get("Record", {}).get("Section", []):
            if section.get("TOCHeading") == "Names and Identifiers":
                for subsection in section.get("Section", []):
                    if subsection.get("TOCHeading") == "Record Description":
                        for info in subsection.get("Information", []):
                            strings = info.get("Value", {}).get("StringWithMarkup", [])
                            ref_number = info.get("ReferenceNumber")
                            source_name = references.get(ref_number, "No Reference")
                            
                            for item in strings:
                                records.append({
                                    "CID": cid,
                                    "String": item.get("String", ""),
                                    "Reference": source_name
                                })
        return records if records else [{"CID": cid, "String": None, "Reference": "No Data Found"}]
    except Exception as e:
        return [{"CID": cid, "String": None, "Reference": f"Error: {str(e)}"}]

def process_cid_record_descriptions(input_file, output_file, delay=0.2):
    """
    Process a CSV file with CIDs to extract Record Description sections and save to a CSV.
    
    Args:
        input_file (str): Path to the input CSV file with a column "CID".
        output_file (str): Path to the output CSV file.
        delay (float): Delay in seconds between each request to comply with throttling guidelines.
    """
    # Load input CSV
    cids = pd.read_csv(input_file)
    if "CID" not in cids.columns:
        raise ValueError("Input CSV must have a column named 'CID'")
    
    # Read processed CIDs from log
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r", encoding="utf-8") as log:
            processed_cids = set(line.strip() for line in log)
    else:
        processed_cids = set()

    # Open output file in append mode
    with open(output_file, "a", encoding="utf-8-sig") as output, open(LOG_FILE, "a", encoding="utf-8") as log:
        for cid in tqdm(cids["CID"], desc="Processing CIDs", unit="CID"):
            # Skip already processed CIDs
            if str(cid) in processed_cids:
                continue
            
            # Fetch data
            records = fetch_record_description(cid)
            
            # Save to output file immediately
            for record in records:
                output.write(f"{record['CID']},\"{record['String']}\",\"{record['Reference']}\"\n")
            
            # Log the processed CID
            log.write(f"{cid}\n")
            log.flush()  # Ensure the log is written immediately
            
            time.sleep(delay)  # Respect throttling limit

    print(f"Record descriptions saved to {output_file}")


input_csv = "input_cid.csv"
output_csv = "pubchem_cid_description.csv"

process_cid_record_descriptions(input_csv, output_csv, delay=0.2)

Processing CIDs: 100%|██████████| 220855/220855 [7:01:11<00:00,  8.74CID/s]   

Record descriptions saved to pubchem_cid_description.csv


In [47]:
pubchem_description = pd.read_csv('pubchem_cid_description.csv', on_bad_lines='skip', names=['CID', 'String', 'Reference'])
pubchem_description

C:\Users\shunn\AppData\Local\Temp\ipykernel_29836\726809557.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pubchem_description = pd.read_csv('pubchem_cid_description.csv', on_bad_lines='skip', names=['CID', 'String', 'Reference'])


,CID,String,Reference
0,1,O-acetylcarnitine is an O-acylcarnitine having...,ChEBI
1,1,Acetylcarnitine is an investigational drug in ...,DrugBank
2,1,Acetylcarnitine has been reported in Drosophil...,LOTUS - the natural products occurrence database
3,1,L-Acetylcarnitine is a metabolite found in or ...,Yeast Metabolome Database (YMDB)
4,1,An acetic acid ester of CARNITINE that facilit...,Medical Subject Headings (MeSH)
...,...,...,...
277345,9999342,NaN,No Data Found
277346,9999516,NaN,No Data Found
277347,9999932,NaN,No Data Found
277348,9999996,NaN,No Data Found


In [55]:
pubchem_description_pivot = pubchem_description.pivot_table(index='CID', columns='Reference', values='String', aggfunc=lambda x: ' | '.join(x.dropna().unique()))
pubchem_description_pivot.reset_index(inplace=True)
pubchem_description_pivot = pubchem_description_pivot[['CID', 'Agency for Toxic Substances and Disease Registry (ATSDR)', 'CAMEO Chemicals', 'California Office of Environmental Health Hazard Assessment (OEHHA)',
                                                      'ChEBI', 'Drug Enforcement Administration (DEA)', 'DrugBank', 'E. coli Metabolome Database (ECMDB)', 'FDA Pharm Classes', 'EPA Air Toxics', 'LOTUS - the natural products occurrence database',
                                                      'LiverTox', 'Medical Subject Headings (MeSH)', 'NCI Thesaurus (NCIt)', 'PubChem', 'Toxin and Toxin Target Database (T3DB)', 'Yeast Metabolome Database (YMDB)']]
pubchem_description_pivot

Reference,CID,Agency for Toxic Substances and Disease Registry (ATSDR),CAMEO Chemicals,California Office of Environmental Health Hazard Assessment (OEHHA),ChEBI,Drug Enforcement Administration (DEA),DrugBank,E. coli Metabolome Database (ECMDB),FDA Pharm Classes,EPA Air Toxics,LOTUS - the natural products occurrence database,LiverTox,Medical Subject Headings (MeSH),NCI Thesaurus (NCIt),PubChem,Toxin and Toxin Target Database (T3DB),Yeast Metabolome Database (YMDB)
0,91,NaN,NaN,NaN,3-(3-hydroxyphenyl)propanoic acid is a monocar...,NaN,NaN,3-(3-Hydroxyphenyl)propanoic acid is a metabol...,NaN,NaN,3-(3-Hydroxyphenyl)propanoic acid has been rep...,NaN,NaN,NaN,NaN,NaN,NaN
1,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93,NaN,NaN,NaN,3-oxoadipic acid is an oxo dicarboxylic acid c...,NaN,NaN,NaN,NaN,NaN,3-Oxoadipic acid has been reported in Rhodococ...,NaN,NaN,NaN,NaN,NaN,NaN
3,96,NaN,NaN,NaN,Acetoacetic acid is a 3-oxo monocarboxylic aci...,NaN,NaN,Acetoacetic acid is a metabolite found in or p...,NaN,NaN,Acetoacetic acid has been reported in Homo sap...,NaN,NaN,NaN,NaN,NaN,NaN
4,98,NaN,NaN,NaN,3-mercaptopyruvic acid is a 2-oxo monocarboxyl...,NaN,NaN,3-Mercaptopyruvic acid is a metabolite found i...,NaN,NaN,3-Mercaptopyruvic acid has been reported in Ho...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220842,90659752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220843,Dimethyl dialkyl ammonium chloride belongs to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R being an alkyl group or an aryl group[1]. (...,NaN
220844,Heptyl 4-hydroxybenzoate belongs to the family...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,which is a benzene ring bearing a carboxylic ...,NaN
220845,Methyl beta-naphthyl ketone belongs to the fam...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"which consists of two fused benzene rings.""",NaN


In [ ]:
bmg_db = biomedgraphica_drug[['BioMedGraphica_ID', 'DrugBank_ID']]
bmg_db['DrugBank_ID'] = bmg_db['DrugBank_ID'].str.split(';')
bmg_db = bmg_db.explode('DrugBank_ID')

drug_description_drugbank = pd.merge(bmg_db, db_description, left_on='DrugBank_ID', right_on='DrugBank ID', how='left')
drug_description_drugbank.drop(columns=['DrugBank ID', 'DrugBank_ID'], inplace=True)
drug_description_drugbank.rename(columns={'Description':'DrugBank'}, inplace=True)
drug_description_drugbank = drug_description_drugbank.groupby('BioMedGraphica_ID').agg({
    'DrugBank': lambda x: ';'.join(x.dropna().unique())
}).reset_index()
drug_description_drugbank.replace('', pd.NA, inplace=True)
drug_description_drugbank

In [66]:
bmg_cid = biomedgraphica_drug[['BioMedGraphica_ID', 'PubChem_CID']]
bmg_cid['PubChem_CID'] = bmg_cid['PubChem_CID'].str.split(';')
bmg_cid = bmg_cid.explode('PubChem_CID')

drug_description_pubchem = pd.merge(bmg_cid, pubchem_description_pivot, left_on='PubChem_CID', right_on='CID', how='left')
drug_description_pubchem.drop(columns=['PubChem_CID', 'CID'], inplace=True)
drug_description_pubchem

C:\Users\shunn\AppData\Local\Temp\ipykernel_29836\2402150304.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmg_cid['PubChem_CID'] = bmg_cid['PubChem_CID'].str.split(';')


,BioMedGraphica_ID,Agency for Toxic Substances and Disease Registry (ATSDR),CAMEO Chemicals,California Office of Environmental Health Hazard Assessment (OEHHA),ChEBI,Drug Enforcement Administration (DEA),DrugBank,E. coli Metabolome Database (ECMDB),FDA Pharm Classes,EPA Air Toxics,LOTUS - the natural products occurrence database,LiverTox,Medical Subject Headings (MeSH),NCI Thesaurus (NCIt),PubChem,Toxin and Toxin Target Database (T3DB),Yeast Metabolome Database (YMDB)
0,BMG_DG000001,NaN,NaN,NaN,O-acetylcarnitine is an O-acylcarnitine having...,NaN,Acetylcarnitine is an investigational drug in ...,NaN,NaN,NaN,Acetylcarnitine has been reported in Drosophil...,NaN,An acetic acid ester of CARNITINE that facilit...,NaN,See also: Acetyl-L-Carnitine (annotation moved...,NaN,L-Acetylcarnitine is a metabolite found in or ...
1,BMG_DG000002,NaN,NaN,NaN,Phenylethanolamine is the simplest member of t...,NaN,NaN,2-Hydroxyphenethylamine is a metabolite found ...,NaN,NaN,2-Amino-1-phenylethanol has been reported in P...,NaN,Simple amine found in the brain. It may be mod...,NaN,NaN,NaN,NaN
2,BMG_DG000003,NaN,"3-chloro-1,1,1-trifluoropropane appears as a c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BMG_DG000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BMG_DG000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273381,BMG_DG273382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273382,BMG_DG273383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273383,BMG_DG273384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273384,BMG_DG273385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 6.3 Final Description

In [67]:
drug_description = pd.merge(drug_description_drugbank, drug_description_pubchem, on='BioMedGraphica_ID', how='outer')
drug_description.replace('', pd.NA, inplace=True)
drug_description.drop_duplicates(inplace=True)
drug_description.reset_index(drop=True, inplace=True)
drug_description

,BioMedGraphica_ID,DrugBank_x,Agency for Toxic Substances and Disease Registry (ATSDR),CAMEO Chemicals,California Office of Environmental Health Hazard Assessment (OEHHA),ChEBI,Drug Enforcement Administration (DEA),DrugBank_y,E. coli Metabolome Database (ECMDB),FDA Pharm Classes,EPA Air Toxics,LOTUS - the natural products occurrence database,LiverTox,Medical Subject Headings (MeSH),NCI Thesaurus (NCIt),PubChem,Toxin and Toxin Target Database (T3DB),Yeast Metabolome Database (YMDB)
0,BMG_DG000001,<NA>,NaN,NaN,NaN,O-acetylcarnitine is an O-acylcarnitine having...,NaN,Acetylcarnitine is an investigational drug in ...,NaN,NaN,NaN,Acetylcarnitine has been reported in Drosophil...,NaN,An acetic acid ester of CARNITINE that facilit...,NaN,See also: Acetyl-L-Carnitine (annotation moved...,NaN,L-Acetylcarnitine is a metabolite found in or ...
1,BMG_DG000002,<NA>,NaN,NaN,NaN,Phenylethanolamine is the simplest member of t...,NaN,NaN,2-Hydroxyphenethylamine is a metabolite found ...,NaN,NaN,2-Amino-1-phenylethanol has been reported in P...,NaN,Simple amine found in the brain. It may be mod...,NaN,NaN,NaN,NaN
2,BMG_DG000003,<NA>,NaN,"3-chloro-1,1,1-trifluoropropane appears as a c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BMG_DG000004,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BMG_DG000005,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273381,BMG_DG273382,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273382,BMG_DG273383,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273383,BMG_DG273384,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273384,BMG_DG273385,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
def merge_string_columns(df, columns, merge_name, separator, split_values=True):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                if split_values:
                    combined.update(row[column].split(separator))
                else:
                    combined.add(row[column])
        return separator.join(combined)

    df[merge_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

drug_description = merge_string_columns(drug_description, ['DrugBank_x', 'DrugBank_y'], 'DrugBank', ' | ', split_values=True)
drug_description.replace('', pd.NA, inplace=True)
drug_description

,BioMedGraphica_ID,Agency for Toxic Substances and Disease Registry (ATSDR),CAMEO Chemicals,California Office of Environmental Health Hazard Assessment (OEHHA),ChEBI,Drug Enforcement Administration (DEA),E. coli Metabolome Database (ECMDB),FDA Pharm Classes,EPA Air Toxics,LOTUS - the natural products occurrence database,LiverTox,Medical Subject Headings (MeSH),NCI Thesaurus (NCIt),PubChem,Toxin and Toxin Target Database (T3DB),Yeast Metabolome Database (YMDB),DrugBank
0,BMG_DG000001,NaN,NaN,NaN,O-acetylcarnitine is an O-acylcarnitine having...,NaN,NaN,NaN,NaN,Acetylcarnitine has been reported in Drosophil...,NaN,An acetic acid ester of CARNITINE that facilit...,NaN,See also: Acetyl-L-Carnitine (annotation moved...,NaN,L-Acetylcarnitine is a metabolite found in or ...,Acetylcarnitine is an investigational drug in ...
1,BMG_DG000002,NaN,NaN,NaN,Phenylethanolamine is the simplest member of t...,NaN,2-Hydroxyphenethylamine is a metabolite found ...,NaN,NaN,2-Amino-1-phenylethanol has been reported in P...,NaN,Simple amine found in the brain. It may be mod...,NaN,NaN,NaN,NaN,<NA>
2,BMG_DG000003,NaN,"3-chloro-1,1,1-trifluoropropane appears as a c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,BMG_DG000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,BMG_DG000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273381,BMG_DG273382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
273382,BMG_DG273383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
273383,BMG_DG273384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
273384,BMG_DG273385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [69]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_Description.csv'
drug_description.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_Description.csv


### 6.4 Combined Description

In [70]:
comb_description = drug_description.copy()

# add the column name at the beginning of the string
# first, we need to get the column names
column_names = comb_description.columns.tolist()
column_names = [col for col in column_names if col != 'BioMedGraphica_ID']
# then we can apply the function to each column
for col in column_names:
    comb_description[col] = comb_description[col].apply(lambda x: ' | '.join([f"{col}: {i}" for i in x.split(' | ')]) if pd.notna(x) else x)

# now we can merge the columns into one
comb_description['Description'] = comb_description[column_names].apply(lambda x: ' | '.join(x.dropna()), axis=1)
comb_description = comb_description[['BioMedGraphica_ID', 'Description']]
comb_description.replace('', pd.NA, inplace=True)
comb_description.drop_duplicates(inplace=True)
comb_description.reset_index(drop=True, inplace=True)
comb_description

,BioMedGraphica_ID,Description
0,BMG_DG000001,ChEBI: O-acetylcarnitine is an O-acylcarnitine...
1,BMG_DG000002,ChEBI: Phenylethanolamine is the simplest memb...
2,BMG_DG000003,"CAMEO Chemicals: 3-chloro-1,1,1-trifluoropropa..."
3,BMG_DG000004,<NA>
4,BMG_DG000005,<NA>
...,...,...
273381,BMG_DG273382,<NA>
273382,BMG_DG273383,<NA>
273383,BMG_DG273384,<NA>
273384,BMG_DG273385,<NA>


In [71]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Drug'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Drug_Description_Combined.csv'
comb_description.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Entity\Drug\BioMedGraphica_Drug_Description_Combined.csv
